# DS 201 C*
See DataStax Academy course here: https://academy.datastax.com/resources/ds201-foundations-apache-cassandra

In [25]:
!pip install cassandra-driver
!pip install names
!pip install RandomWords

You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 51kB 733kB/s ta 0:00:011
  Running setup.py bdist_wheel for RandomWords ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/0b/15/80/e91c07dde49e787cc1d00ad0bbb90d3368322d4a29c3961043
Successfully built RandomWords
You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [31]:
from cassandra.cluster import Cluster
import uuid
import names
from random_words import RandomWords
import random
import string

Connect to the cluster. The cluster should be run using Docker with the following:
```
docker run -p 9042:9042 -p 9160:9160 --name cassandra -d cassandra:latest
```

In [6]:
# cluster = Cluster(['127.0.0.1'])
cluster = Cluster(['cassandra-node'])
session = cluster.connect()

Generate the `killrvideo` keyspace

In [10]:
session.execute(
    """
    CREATE KEYSPACE killrvideo WITH REPLICATION = {
        'class': 'SimpleStrategy',
        'replication_factor': 1
    };
    """
)

Set keyspace to `killrvideo` keyspace

In [11]:
session.set_keyspace('killrvideo')

Generate tables

In [20]:
session.execute(
    """
    CREATE TABLE users (
        user_id UUID,
        first_name TEXT,
        last_name TEXT,
        PRIMARY KEY (user_id)
        );
    """
)

In [13]:
session.execute(
    """
    CREATE TABLE movies_by_actor (
        actor TEXT,
        release_year INT,
        movie_id UUID,
        title TEXT,
        genres set<TEXT>,
        rating FLOAT,
        PRIMARY KEY ((actor), release_year, movie_id)
        ) WITH CLUSTERING ORDER BY (release_year DESC, movie_id ASC);
    """
)

Add some data to our table

In [49]:
def gen_actor():
    rw = RandomWords()
    letter = random.choice('qwertyuiopasdfghjklzcvbnm')
    genres = ['Action','Adventure','Comedy','Crime','Drama','Fantasy','Historical','Horror','Mystery','Romance','SciFi']
    num_genres = random.randint(1,3)
    
    actor = names.get_full_name()
    release_year = random.randint(1970,2018)
    movie_id = uuid.uuid1()
    title = ' '.join(rw.random_words(letter=letter, count=3)).title()
    genres = set(random.sample(genres, num_genres))
    rating = random.randint(1,5)
    
    return (actor, release_year, movie_id, title, genres, rating)    

In [50]:
for i in range(1000):
    actor = gen_actor()
    
    session.execute(
        """
        INSERT INTO movies_by_actor (actor, release_year, movie_id, title, genres, rating)
        VALUES (%s, %s, %s, %s, %s, %s)
        """,
        actor
    )    

In [71]:
session.execute(
    """
    INSERT INTO users (user_id, first_name, last_name)
    VALUES (%s, %s, %s)
    """,
    (uuid.uuid1(), "Bilbo", "Baggins")
)

We can query our keyspace to demonstrate that the data entered the DB:

In [51]:
results = session.execute(
    """
    select * from movies_by_actor limit 10
    """
)

for row in results:
    print(row)

Row(actor='Steven Vasquez', release_year=1998, movie_id=UUID('54cf878e-2e17-11e8-a7c7-e97a3c3d48a2'), genres=SortedSet(['Drama']), rating=1.0, title='Quarts Qualifiers Quota')
Row(actor='Julie Surber', release_year=2003, movie_id=UUID('535b244e-2e17-11e8-a7c7-e97a3c3d48a2'), genres=SortedSet(['Romance']), rating=5.0, title='Mules Models Mosses')
Row(actor='Lyndon Johnson', release_year=2001, movie_id=UUID('561fbafa-2e17-11e8-a7c7-e97a3c3d48a2'), genres=SortedSet(['SciFi']), rating=1.0, title='Losses Lifeboat Latitude')
Row(actor='Michael Brown', release_year=1997, movie_id=UUID('540d1f8c-2e17-11e8-a7c7-e97a3c3d48a2'), genres=SortedSet(['Adventure']), rating=5.0, title='Recruits Rebound Rams')
Row(actor='David Limon', release_year=1998, movie_id=UUID('5595fcac-2e17-11e8-a7c7-e97a3c3d48a2'), genres=SortedSet(['Fantasy', 'Horror']), rating=4.0, title='Quarterdecks Quart Quiet')
Row(actor='Katherine Hickman', release_year=2000, movie_id=UUID('56b461a0-2e17-11e8-a7c7-e97a3c3d48a2'), genres=

We can also delete this record if we so choose:

In [57]:
deleteStatement = """
    DELETE FROM killrvideo.users
    WHERE user_id = {0};
"""

session.execute(deleteStatement.format(row.user_id))